<a href="https://colab.research.google.com/github/bioimage-informatics/restworm_cloud/blob/main/restworm_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# streamlit install
!pip install --upgrade streamlit

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
# Original
%%writefile /content/drive/MyDrive/Web_app/app.py

# Libraries
import keras
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import cv2
from network import RESTWORM_NET
from PIL import Image
import time

# Parameters - single GPU -

# model architecture
input_image_size = 256
input_channel_count = 1
output_channel_count = 1
first_layer_filter_count = 64

# Early stopping
es='No'
# data augmentation
aug='No'
# optimizer, loss function
optimizer = 'adam'
loss = 'mse'
# Computation size parameters
BATCH_SIZE = 4
NUM_EPOCH = 5
num_layers = 4
maxPool = True
act = 'None'
ROW_SIZE = 256
COLUME_SIZE = 256

# Title
st.title("restworm cloud")
# st.write("streamlit")


# Sidebar
st.sidebar.markdown("### Upload an image to restore")
# Upload a file
uploaded_files = st.sidebar.file_uploader("Choose an image file", accept_multiple_files= False)

# Directory
# https://github.com/aidanjungo/StreamlitDirectoryPicker

# If an image file is uploaded
if uploaded_files:
    # Display uploaded image
    image=Image.open(uploaded_files)
    image = np.array(image)
    image_for_view = image / np.max(image)
    st.image(image_for_view, caption='Uploaded image')
    
    # Run button
    execute_pairplot = st.button("Run restoration")
    
    # If run button pushed
    if execute_pairplot:
        # !wget -P "/content/drive/My Drive/Web_app/" https://xxxxx.xxx
        filename = "/content/drive/MyDrive/Web_app/model"
        model= keras.models.load_model(filename, compile=False)

        # Progress bar
        my_bar = st.progress(0)

        # input test image
        x_test = np.zeros((1, COLUME_SIZE, ROW_SIZE, 1), np.float32)
        my_bar.progress(0.1)

        # image information
        if image.ndim==3:
            image  = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        input_row, input_column = image.shape
        image = cv2.resize(image, [ROW_SIZE, COLUME_SIZE])
        x_test[0,:,:,0] = image
        my_bar.progress(0.2)

        # prediction
        y_pred = model.predict(x_test, BATCH_SIZE)

        # Display restored image
        my_bar.progress(0.95)
        y_pred_input_size = cv2.resize(y_pred[0,:,:,0], [input_row, input_column])
        pred_img = y_pred_input_size / np.max(y_pred_input_size)
        my_bar.progress(1.0)
        st.image(pred_img, caption='Restored image',clamp=True)




In [ ]:
# Run file browser, open app.py on an editer
from google.colab import files
files.view("/content/drive/MyDrive/Web_app/")
files.view("/content/drive/MyDrive/Web_app/app.py")
# Run Streamlit through a localtunnel
!streamlit run /content/drive/MyDrive/Web_app/app.py & sleep 3 && npx localtunnel --port 8501